# JPA 연관 관계 정리

## 서론. JPA 를 사용하는 이유
- "객체와 관계형 데이터베이스 테이블이 어떻게 매핑되는지를 이해하는 것"

 -> 객체 지향 프로그래밍과 데이터베이스 사이의 패러다임 불일치를 해결

### 연관 관계 정의 규칙

- 방향 : 단방향, 양방향 (객체 참조)
- 연관 관계의 주인 : 양방향일 때, 연관 관계에서 관리 주체
- 다중성 : 다대일(N:1), 일대다(1:N), 일대일(1:1), 다대다(N:M)


## 1. 방향

### 단방향, 양방향
데이터베이스 테이블은 외래 키 하나로 양 쪽 테이블 조인이 가능

따라서 데이터베이스는 단방향 양방향 개념이 없음

하지만 객체는 참조용 필드가 따로 존재해야함 -> 두 객체 사이에 하나의 객체만 참조용 필드를 갖고 참조하면 단방향, 두 객체가 모두 각각 참조용 필드를 갖고 참조하면 양방향 관계

### 무조건 양방향 관계를 가지지 않는 이유
#### 객체가 무조건 양방향 관계를 가질 경우
예를 들어 하나의 객체가 여러 개의 객체들과 관계를 가지고 있다고 하면 그 객체는 굉장히 더러워 질것이다

또한 그 객체와 연관되있는 객체들을 매핑할 때 불필요한 객체들도 같이 매핑되므로 복잡성이 훨씬 증가함

## 2. 연관 관계의 주인
두 객체(A, B)가 양방향 관계, 다시 말해 단방향 관계 2개(A→B, B→A)를 맺을 때, 연관 관계의 주인을 지정해야 합니다.

연관 관계의 주인을 지정 하는 것은 두 단방향 관계(A→B, B→A)중, 제어의 권한(외래 키를 비롯한 테이블 레코드를 저장, 수정, 삭제 처리)을 갖는 실질적인 관계가 어떤 것인지 JPA에게 알려준다고 생각하면 됩니다.

연관 관계의 주인은 연관 관계를 갖는 두 객체 사이에서 조회, 저장, 수정, 삭제를 할 수 있지만, 연관 관계의 주인이 아니면 조회만 가능합니다.

연관 관계의 주인이 아닌 객체에서 mappedBy 속성을 사용해서 주인을 지정해줘야합니다.

TIP : 외래 키가 있는 곳을 연관 관계의 주인으로 정하면 됩니다. 무조건


### 연관 관계의 주인을 맺어주는 이유
양방향 연관 관계 관리 포인트가 두 개일 때는 테이블과 매핑을 담당하는 JPA입장에서 혼란을 준다.

A와 B가 양방향 관계일 경우 A의 FK 를 수정해야할지 B의 FK를 수정해야할지 결정해야하기 때문!

### 그렇다면 연관관계 주인만 제어?
두 참조를 사용하는 순수한 두 객체는 **데이터 동기화**를 해줘야하기 때문에 둘 다 변경하는 것이 좋음

## 3.다중성

### 데이터베이스를 기준으로 다중성을 결정

(JPA는 JPQL도 그렇고 보통 객체를 기준으로 하는게 일반적인데 다중성을 정하는 기준은 데이터베이스 기준인게 신기합니다.)

연관 관계는 대칭성을 갖습니다.
- 일대다 ↔ 다대일
- 일대일 ↔ 일대일
- 다대다 ↔ 다대다


### 다대일(N:1)

#### 다대일(N:1) 단방향
주요 키워드 
- @ManyToOne
- Post : 주인 | Board : 하위
```java
@Entity 
public class Post { 
    @Id 
    @GeneratedValue 
    @Column(name = "POST_ID") 
    private Long id; 

    @Column(name = "TITLE")
    private String title; 

    @ManyToOne 
    @JoinColumn(name = "BOARD_ID")
    private Board board; 
    //... getter, setter
}
@Entity 
public class Board { 
    @Id 
    @GeneratedValue 
    private Long id; 
    private String title;
    //... getter, setter
}
```

#### 다대일(N:1) 양방향
주요 키워드 
- @ManyToOne & @OneToMany(mappedBy = ?)

양방향 매핑을 사용하면 연관관계의 주인을 설정해주어야함 : (mappedBy = ?)

- Post : 주인 | Board : 하위
```java
@Entity 
public class Post { 
    @Id 
    @GeneratedValue 
    @Column(name = "POST_ID") 
    private Long id;
    
    @Column(name = "TITLE") 
    private String title; 
    
    @ManyToOne @JoinColumn(name = "BOARD_ID") 
    private Board board;
    //... getter, setter 
} 
@Entity 
public class Board { 
    @Id 
    @GeneratedValue 
    private Long id; 
    private String title;
    
    @OneToMany(mappedBy = "board") 
    List<Post> posts = new ArrayList<>(); 
    //... getter, setter 
}
```

### 일대일(1:1)

#### 일대일(1:1) 단방향
주요 키워드 
- @OneToOne

- Post : 주인 | Board : 하위
```java
@Entity
public class Post { 
    @Id 
    @GeneratedValue 
    @Column(name = "POST_ID") 
    private Long id;
    
    @Column(name = "TITLE") 
    private String title; 
    
    @OneToOne 
    @JoinColumn(name = "ATTACH_ID") 
    private Attach attach; 
    //... getter,setter 
} 
@Entity 
public class Attach { 
    @Id 
    @GeneratedValue
    @Column(name = "ATTACH_ID")
    private Long id; 
    private String name; 
    //... getter, setter 
}
```

#### 일대일(1:1) 양방향
주요 키워드 
- @OneToOne(mappedBy = "attach") 

양방향 매핑을 사용하면 연관관계의 주인을 설정해주어야함 : (mappedBy = ?)

- Post : 주인 | Board : 하위
```java
@Entity
public class Post { 
    @Id 
    @GeneratedValue 
    @Column(name = "POST_ID") 
    private Long id;
    
    @Column(name = "TITLE") 
    private String title; 
    
    @OneToOne 
    @JoinColumn(name = "ATTACH_ID") 
    private Attach attach; 
    //... getter,setter 
} 
@Entity 
public class Attach { 
    @Id 
    @GeneratedValue
    @Column(name = "ATTACH_ID")
    private Long id; 
    private String name;
    
    @OneToOne(mappedBy = "attach") 
    private Post post;
    //... getter, setter 
}
```